In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('processed_data.csv')
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

In [4]:
data.drop(columns={'Unnamed: 0'}, inplace=True)

In [6]:
from sklearn.utils import shuffle

In [7]:
data = shuffle(data, random_state=42).reset_index(drop=True)
data

,target,text
0,4,okno growing purple againor black blueand dark...
1,4,morning
2,4,hot pockets 4am nice
3,0,wanted concert
4,4,glad finished portfolio preface ah i'm proud
...,...,...
1589596,0,joe said best concert sao paulo true know
1589597,4,taking shower
1589598,0,watched weeks gossip girl omg can't wait 4 wee...
1589599,0,watchmoviesnet messed


In [8]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

2024-10-29 21:56:28.887183: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
df = data

In [10]:
from sklearn.model_selection import train_test_split


In [11]:
# Convert 'target' to binary sentiment labels (0 or 1)
df['target'] = df['target'].apply(lambda x: 0 if x == 0 else 1)

# Split data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert data to TensorFlow datasets
train_data = tf.data.Dataset.from_tensor_slices((train_df['text'].values, train_df['target'].values))
test_data = tf.data.Dataset.from_tensor_slices((test_df['text'].values, test_df['target'].values))


In [17]:
train_df.shape

(1271680, 2)

In [12]:
bert_model_url = "https://tfhub.dev/google/experts/bert/wiki_books/sst2/2"  # SST-2 model trained for sentiment analysis
preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

bert_preprocessor = hub.KerasLayer(preprocessor_url)
bert_encoder = hub.KerasLayer(bert_model_url)


In [13]:
def build_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessed_text = bert_preprocessor(text_input)
    outputs = bert_encoder(preprocessed_text)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)
    return tf.keras.Model(inputs=[text_input], outputs=[net])

model = build_model()


In [14]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [18]:
# Original shape
original_shape = df.shape
print("Original DataFrame shape:", original_shape)

# Calculate the new size (1/100 of the original)
new_size = original_shape[0] // 100  # integer division to get the whole number
print("New size for training data:", new_size)

# Randomly sample the training data
smaller_train_df = train_df.sample(n=new_size, random_state=42)

# Check the shape of the new training dataset
print("Smaller Training DataFrame shape:", smaller_train_df.shape)

smaller_train_data = tf.data.Dataset.from_tensor_slices((smaller_train_df['text'].values, smaller_train_df['target'].values))

Original DataFrame shape: (1589601, 2)
New size for training data: 15896
Smaller Training DataFrame shape: (15896, 2)


In [19]:
BATCH_SIZE = 32
train_data = smaller_train_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

history = model.fit(train_data, validation_data=test_data, epochs=3)


Epoch 1/3


364/497 [====================>.........] - ETA: 14:17 - loss: 0.5603 - accuracy: 0.7171

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy:.2f}")
